In [1]:
import json
import os
import sys
import cProfile
import pstats

import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error

sys.path.append(r"../")
from AssistanceTransform import transform, estimator

# Performance
In this Notebook, I will be performance testing various parts of the AssistanceTransform library. This includes both speed and accuracy testing.

First getting the testing data ready:

In [2]:
image_coord_dict = dict()
image_coord_dict["00.jpg"] = np.array([[1470, 1430], [2333, 1355], [3247, 1781], [1935, 1952]])
image_coord_dict["01.jpg"] = np.array([[1495, 1552], [2219, 1589], [1843, 1969], [805, 1875]])
image_coord_dict["03.jpg"] = np.array([[1216, 1398], [2215, 1754], [3268, 1530], [2067, 1282]])   

def setup_vars():
    """Loads data for test_transform_image"""
    data_dir = r"./data/table"
    json_fp = os.path.join(data_dir, "anno.json")
    arr_fp = os.path.join(data_dir, "anno.npz")
    with open(json_fp, "r") as fp:
        mappings = json.load(fp)

    with np.load(arr_fp) as arrs:
        anno_dict = {img: {"heads": arrs[f"{prefix}heads"],
                           "feet": arrs[f"{prefix}feet"]}
                     for img, prefix in mappings.items()}
    
    for key, items in anno_dict.items():
        if key.endswith("02.jpg"):
            continue
        else:
            image_coords = image_coord_dict[key[-6:]]
        # feet and heads have been swapped in annotations
        reference = items["feet"], items["heads"]
        height = 0.095  # m
        STD = 0.01  # m
        img = Image.open(key)
        yield (img, reference, height, STD, image_coords)

## Speed

In [3]:
generator = setup_vars()
pr = cProfile.Profile()
pr.enable()
coords_sq = (transform.transform_image(*params, iters=1e5)[:, :2] for params in generator)
y_pred = np.array([estimator.area(poly) for poly in coords_sq])
pr.disable()
pr.print_stats("tottime")

C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
100%|█████████████████████████████████████████| 100000/100000 [01:06<00:00, 1500.90it/s, acc_rate=0.425, factor=0.0515]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|▏                                             | 275/100000 [00:00<01:11, 1399.81it/s, a

       elevation_m   tilt_deg  heading_deg  roll_deg  probability
0         0.306243  81.996112   -79.653107  1.629381    12.729688
1         0.344094  82.007254   -79.709764  1.716676    12.456890
2         0.299166  81.960093   -79.673966  1.666868    12.114685
3         0.299166  81.960093   -79.673966  1.666868    12.114685
4         0.299166  81.960093   -79.673966  1.666868    12.114685
...            ...        ...          ...       ...          ...
89994     0.405648  80.044922   -76.616153  4.984297    13.368879
89995     0.419810  80.053144   -76.699074  4.939615    13.465128
89996     0.419810  80.053144   -76.699074  4.939615    13.465128
89997     0.411726  80.098583   -76.727362  4.938947    13.461845
89998     0.425388  79.993010   -76.793508  4.899426    13.474256

[89999 rows x 5 columns]


100%|███████████████████████████████████████████| 100000/100000 [01:02<00:00, 1598.11it/s, acc_rate=0.46, factor=0.127]
C:\Users\Caspe\Anaconda3\envs\AssistanceTransform\lib\site-packages\cameratransform\camera.py:952: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  given = np.array([X, Y, Z])
  0%|                                               | 178/100000 [00:00<00:56, 1762.66it/s, acc_rate=0.09, factor=0.81]

       elevation_m   tilt_deg  heading_deg  roll_deg  probability
0         0.836337  72.071649   -79.454563  7.074569    13.744670
1         0.836337  72.071649   -79.454563  7.074569    13.744670
2         0.852193  71.913912   -79.484240  7.079909    13.773489
3         0.852193  71.913912   -79.484240  7.079909    13.773489
4         0.878754  71.808840   -79.374844  6.763359    13.731037
...            ...        ...          ...       ...          ...
89994     1.233127  61.804541   -69.684293  9.123352    12.528525
89995     1.154303  61.795314   -69.688130  9.211771    12.370629
89996     1.154303  61.795314   -69.688130  9.211771    12.370629
89997     1.261849  61.662532   -69.450646  9.402856    12.244568
89998     1.261849  61.662532   -69.450646  9.402856    12.244568

[89999 rows x 5 columns]


100%|██████████████████████████████████████████| 100000/100000 [01:03<00:00, 1574.15it/s, acc_rate=0.53, factor=0.0927]


...

     1683    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
       45    0.000    0.000    0.000    0.000 inspect.py:1799(_signature_bound_method)
       93    0.000    0.000    0.000    0.000 base.py:4079(__getitem__)
      174    0.000    0.000    0.000    0.000 base.py:413(find)
     72/8    0.000    0.000    0.000    0.000 ast.py:64(_convert)
       18    0.000    0.000    0.006    0.000 format.py:1103(format_array)
      590    0.000    0.000    0.000    0.000 sre_parse.py:233(__next)
      6/3    0.000    0.000    0.001    0.000 base.py:289(__new__)
        3    0.000    0.000    0.006    0.002 parameter_set.py:195(set_to_mean)
       24    0.000    0.000    0.002    0.000 series.py:201(__init__)
       15    0.000    0.000    0.014    0.001 _distn_infrastructure.py:417(__init__)
      135    0.000    0.000    0.001    0.000 {pandas._libs.lib.is_list_like}
       21    0.000    0.000    0.002    0.000 indexing.py:757(_getitem_lowerdim)
      249    0.000    0

       45    0.000    0.000    0.000    0.000 _methods.py:53(_any)
        4    0.000    0.000    0.000    0.000 {built-in method _abc._abc_init}
       45    0.000    0.000    0.000    0.000 TiffImagePlugin.py:643(<lambda>)
       16    0.000    0.000    0.000    0.000 zipfile.py:1023(_read2)
        9    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:523(_compile_bytecode)
        9    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap_external>:1240(_get_spec)
       30    0.000    0.000    0.000    0.000 indexing.py:1419(_validate_integer)
        3    0.000    0.000    0.000    0.000 {built-in method pandas._libs.missing.isnaobj}
      375    0.000    0.000    0.000    0.000 inspect.py:2835(parameters)
      153    0.000    0.000    0.000    0.000 enum.py:628(value)
       12    0.000    0.000    0.000    0.000 base.py:1032(__iter__)
        3    0.000    0.000    0.001    0.000 format.py:1049(_get_formatted_index)
       24    0.000    0.000   

        3    0.000    0.000    0.000    0.000 threading.py:335(notify)
       45    0.000    0.000    0.000    0.000 _util.py:323(<listcomp>)
       45    0.000    0.000    0.000    0.000 _util.py:339(<dictcomp>)
        3    0.000    0.000    0.005    0.002 construction.py:60(arrays_to_mgr)
        3    0.000    0.000    0.002    0.001 threading.py:834(start)
       36    0.000    0.000    0.000    0.000 common.py:603(<genexpr>)
        6    0.000    0.000    0.000    0.000 common.py:218(asarray_tuplesafe)
        1    0.000    0.000    0.001    0.001 fractions.py:4(<module>)
        8    0.000    0.000    0.000    0.000 format.py:282(descr_to_dtype)
        1    0.000    0.000    0.000    0.000 zipfile.py:252(_EndRecData)
       47    0.000    0.000    0.000    0.000 sre_parse.py:286(tell)
       66    0.000    0.000    0.000    0.000 _binary.py:21(i8)
        3    0.000    0.000    0.000    0.000 Image.py:530(__init__)
       16    0.000    0.000    0.000    0.000 zipfile.py:940(_up

        8    0.000    0.000    0.000    0.000 tokenize.py:219(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:1216(getLogger)
        6    0.000    0.000    0.000    0.000 generic.py:447(_info_axis)
        3    0.000    0.000    0.000    0.000 concat.py:567(<listcomp>)
       16    0.000    0.000    0.000    0.000 {method 'seekable' of '_io.BufferedReader' objects}
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        3    0.000    0.000    0.000    0.000 hooks.py:103(__call__)
        6    0.000    0.000    0.000    0.000 Image.py:3053(register_mime)
        3    0.000    0.000    0.000    0.000 utils.py:255(_supports_unicode)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-6437586f9fa3>:6(<module>)
       12    0.000    0.000    0.000    0.000 config.py:587(_get_registered_option)
        3    0.000    0.000    0.000    0.000 std.py:536(<genexpr>)
        3    0.000    0.000    0.000    0.000 JpegImagePlu

        3    0.000    0.000    0.000    0.000 base.py:3101(_get_partial_string_timestamp_match_key)
        3    0.000    0.000    0.000    0.000 format.py:1052(<dictcomp>)
        3    0.000    0.000    0.000    0.000 TiffImagePlugin.py:332(denominator)
        5    0.000    0.000    0.000    0.000 TiffImagePlugin.py:617(_register_loader)
        3    0.000    0.000    0.000    0.000 interactiveshell.py:3318(compare)
        1    0.000    0.000    0.000    0.000 zipfile.py:1397(<listcomp>)
        3    0.000    0.000    0.000    0.000 indexing.py:913(_validate_key)
        1    0.000    0.000    0.000    0.000 PpmImagePlugin.py:47(PpmImageFile)
        6    0.000    0.000    0.000    0.000 {built-in method builtins.vars}
        1    0.000    0.000    0.000    0.000 __init__.py:1170(append)
        4    0.000    0.000    0.000    0.000 typing.py:615(<genexpr>)
        1    0.000    0.000    0.000    0.000 npyio.py:206(__exit__)
        3    0.000    0.000    0.000    0.000 base.py:557

## Accuracy
As the "real" positions in images currently aren't known, in order to figure out how close CameraTransform is to "real" positions, we have to take a derivative of these positions. For the purposes of this assignment, I've chosen the area of the polygon, as this is an important variable in the use case.

The three images used for testing all feature a table with a area of `133` cm$^{2}$. 

In [4]:
y_true = np.repeat(1.33455, y_pred.size)

In [5]:
y_pred

array([1.38932653, 1.59045222, 1.57226316])

In [6]:
mean_squared_error(y_true, y_pred, squared=False)

0.20411921518030426

An RMSE of around 20 square centimeters is adequate. Especially the latter two images are taken and worse angles and are insufficiently labeled in comparison to `img_03.jpg`.